<a href="https://colab.research.google.com/github/lourencocavalcante/Neurocomputacao/blob/main/CAP351_Projeto_01_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<img src="https://github.com/lourencocavalcante/LogosINPE/blob/main/logoinpe.png?raw=true" align="middle">](https://www.gov.br/inpe/pt-br)

[<center><img src="https://github.com/lourencocavalcante/LogosINPE/blob/main/LogoCAP.png?raw=true" width = 500 align="middle"></center>](http://www.inpe.br/posgraduacao/cap/)


---


# **CAP-351 Neurocomputação**

**Professor:**

* Dr. Marcos Goncalves Quiles

**Discente:**
* Lourenço José Cavalcante Neto

# PROPOSTA DE PROJETO 1:


# REDE MLP:: Multi-layer Perceptron (MLP)


1. Selecionar dois datasets (não triviais)

  * Um dataset para classificação
  * Um dataset para regressão
  * Separar em treino/validação/teste

2. Treinar modelos MLP para os dois problemas

3. Considerar:

  * Diferentes topologias (>=5 topologias, variar número de
camadas)

  * Usar o algoritmo original SGD (não usar algoritmos otimizados,
e.g. ADAM)
  
  * Avaliar o impacto do uso do Momentum

  * Avaliar o impacto do uso da regularização (i.e. L2)

4. Ilustrar graficamente a evolução do treinamento (treino/validação).

5. Confeccionar um relatório (reprodutível) contendo os
experimentos e resultados

In [6]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.8.2


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Dataset**

Importando os dados do **disdrômetro RD80 (Joss**) e do radiômetro de microondas **MP3000A**.


In [10]:
#CRIANDO OS DATAFRAMES
#Dados do disdrômetro RD80. Chamado comumente de Joss. 
#Este equipamento mede adistribuição das gotas de chuva que chegam à superfície. (df_rd80_joss)
df_rd80_joss = pd.read_table('/content/drive/MyDrive/MestradoCAP/Lourenco/cap351/Datasets/JWD_campina_report_RZLwcDmDtNdrop_b2.1_report.txt',header=None,delim_whitespace=True)


#Dados do radiômetro de microondas MP3000A. Esse instrumento mede diversas variáveis:

#Variáveis de ambiente (dados_ambi_superficie)
df_mp3000A_1 = pd.read_table('/content/drive/MyDrive/MestradoCAP/Lourenco/cap351/Datasets/level1_output_sfc.txt',header=None,delim_whitespace=True)
#Radiância no espectro de onda do microonda em diversos canais (dados_med_canais_micro)
df_mp3000A_2 = pd.read_table('/content/drive/MyDrive/MestradoCAP/Lourenco/cap351/Datasets/level1_output_bt.txt',header=None,delim_whitespace=True)
#Estimativas de parâmetros de nuvens (dados_est_param_nuvens)
df_mp3000A_3 = pd.read_table('/content/drive/MyDrive/MestradoCAP/Lourenco/cap351/Datasets/level2_output_cld_atto.txt',header=None,delim_whitespace=True)

In [12]:
#Os Dataframes estão sem título nas colunas. Vamos adicioná-las.

df_rd80_joss.columns = ['Ano (YYYY)', 'Mes', 'dia', 'hora', 'minuto', 'segundos','Rain Intensity (mm/h)', 'radar reflectivity (1/mm6m3)','Liquid watercontent (g/m3)',
'Mean weight diameter(mm)', 'Time integration (s)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)', 'N(D)']

df_mp3000A_1.columns = ['Mes', 'dia', 'Ano (YY)', 'Hora', 'Minuto', 'segundos', 'Tamb(K)', 'Rh(%)', 'Pres(mb)', 'Tir(K)', 'Rain', 'DataQuality']
df_mp3000A_2.columns = ['Mes', 'dia', 'Ano (YY)', 'Hora', 'Minuto', 'segundos', 'Ch 22.234', 'Ch 22.500', 'Ch 23.034', 'Ch 23.834', 'Ch 25.000', 'Ch 26.234', 'Ch 28.000', 'Ch 30.000', 'Ch 51.248', 'Ch 51.760', 'Ch 52.280', 'Ch 52.804', 'Ch 53.336', 'Ch 53.848', 'Ch 54.400', 'Ch 54.940', 'Ch 55.500', 'Ch 56.020', 'Ch 56.660', 'Ch 57.288', 'Ch 57.964', 'Ch 58.800', 'DataQuality']
df_mp3000A_3.columns = ['Mes', 'dia', 'Ano (YY)', 'Hora', 'Minuto', 'segundos', 'Int. Vapor(cm)','Int. Liquid(mm)','Cloud Base(km)','DataQuality']

In [17]:
#Modificanto o tipo de dado das colunas referentes a Data e Horário para Inteiro
df_rd80_joss[['Ano (YYYY)', 'Mes', 'dia', 'hora', 'minuto', 'segundos']] = df_rd80_joss[['Ano (YYYY)', 'Mes', 'dia', 'hora', 'minuto', 'segundos']].astype(int)
df_mp3000A_3[['Mes', 'dia', 'Ano (YY)', 'Hora', 'Minuto', 'segundos']] = df_mp3000A_3[['Mes', 'dia', 'Ano (YY)', 'Hora', 'Minuto', 'segundos']].astype(int)
df_mp3000A_2[['Mes', 'dia', 'Ano (YY)', 'Hora', 'Minuto', 'segundos']] = df_mp3000A_2[['Mes', 'dia', 'Ano (YY)', 'Hora', 'Minuto', 'segundos']].astype(int)
df_mp3000A_3[['Mes', 'dia', 'Ano (YY)', 'Hora', 'Minuto', 'segundos']] = df_mp3000A_3[['Mes', 'dia', 'Ano (YY)', 'Hora', 'Minuto', 'segundos']].astype(int)

In [ ]:
for index, row in df_mp3000A_3.iterrows():
    print(len(str(int(row['dia']))))
  
    df_mp3000A_3.loc[index, 'Data'] = str(int(row['dia'])) + '-' + str(int(row['Mes'])) + '-'+ str(int(row['Ano (YY)']))
    df_mp3000A_3.loc[index, 'Horario'] = str(int(row['Hora'])) + ':' + str(int(row['Minuto'])) + ':'+ str(int(row['segundos']))